In [1]:
import pandas as pd
from path import Path
file_path = Path("./Resources/loans_data_encoded.csv")
loans_df = pd.read_csv(file_path)
loans_df.head()

,amount,term,age,bad,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,0,6,0,1,0,0,0,1
1,1000,30,50,0,7,1,0,0,0,1,0
2,1000,30,33,0,8,1,0,0,0,1,0
3,1000,15,27,0,9,0,0,0,1,0,1
4,1000,30,28,0,10,0,0,0,1,1,0


# Split the Dataset

In [4]:
X = loans_df.copy()
X = X.drop("bad", axis=1)
y = loans_df["bad"].values

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1)

# Scaling the dataset

In [8]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train_scaled, y_train.ravel())
   print("Learning rate: ", learning_rate)
   print("Accuracy score (training): {0:.3f}".format(
           classifier.score(X_train_scaled,y_train)))
   print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(X_test_scaled,y_test)))

Learning rate:  0.05
Accuracy score (training): 0.629
Accuracy score (validation): 0.512
Learning rate:  0.1
Accuracy score (training): 0.656
Accuracy score (validation): 0.520
Learning rate:  0.25
Accuracy score (training): 0.723
Accuracy score (validation): 0.536
Learning rate:  0.5
Accuracy score (training): 0.755
Accuracy score (validation): 0.560
Learning rate:  0.75
Accuracy score (training): 0.781
Accuracy score (validation): 0.520
Learning rate:  1
Accuracy score (training): 0.797
Accuracy score (validation): 0.472


A model that performs well on the training set but poorly on the testing set is said to be “overfit.” Overfitting is akin to memorizing the answers to an exam: It will help on that particular exam, but not on any others. In other words, overfitting occurs when a model gives undue importance to patterns within a particular dataset that are not found in other, similar datasets. Instead of learning a general pattern that can be applied to other similar datasets, it learns the patterns specific to one dataset.

0.5 learning rate has the highest validation Accuracy score. Therefore, we use it for the prediction

In [20]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.56


In [22]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,49,16
Actual 1,39,21


In [23]:
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.56      0.75      0.64        65
           1       0.57      0.35      0.43        60

    accuracy                           0.56       125
   macro avg       0.56      0.55      0.54       125
weighted avg       0.56      0.56      0.54       125

